In [34]:
import requests

# API endpoint
url = "http://127.0.0.1:8000/find-image/"

# Request payload
payload = {
    "description": "A silver Tesla Cybertruck on the highway",  # Replace with your desired description
    "nimages": 5  # Number of images to analyze
}

# Request headers
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)


In [35]:
response

<Response [200]>

In [ ]:

# Check the response
if response.status_code == 200:
    print("Selected Image Index:", response.json())
else:
    print("Error:", response.status_code, response.text)


In [3]:
import os
import requests

num_images = 5
query = "A silver Tesla Cybertruck on the highway"

In [4]:
BING_API_KEY = os.getenv("BING_API_KEY")

search_url = "https://api.bing.microsoft.com/v7.0/images/search"
headers = {"Ocp-Apim-Subscription-Key": BING_API_KEY}
params = {"q": query, "count": num_images}

response = requests.get(search_url, headers=headers, params=params)

In [5]:
response

<Response [200]>

In [6]:
results = response.json()
image_urls = [img["contentUrl"] for img in results.get("value", [])[:num_images]]
[print(url) for url in image_urls]

https://static1.hotcarsimages.com/wordpress/wp-content/uploads/2023/04/cybertruck.jpg
https://driveteslacanada.ca/wp-content/uploads/2019/12/Cybertruck-spotted.jpeg
https://i.kinja-img.com/image/upload/c_fill,h_675,pg_1,q_80,w_1200/a552146a9aa390bea912ba0b3cca99e0.jpg
https://s.yimg.com/ny/api/res/1.2/kM_ant_AFlqQAlYEszJCag--/YXBwaWQ9aGlnaGxhbmRlcjt3PTk2MDtoPTY4NA--/https://media.zenfs.com/en/business_insider_articles_888/136f90c113e2054d7fd2ac287f875d7a
https://www.carscoops.com/wp-content/uploads/2023/02/Tesla-Cybertruck.jpg


[None, None, None, None, None]

In [7]:
images = []
for url in image_urls:
    try:
        img_data = requests.get(url).content
        images.append(img_data)
    except Exception:
        continue  # Skip if the image can't be downloaded

In [8]:
import base64
def convert_to_base64(image_data: bytes) -> str:
    return base64.b64encode(image_data).decode("utf-8")



In [9]:
# Step 2: Convert images to base64
images_base64 = [convert_to_base64(img) for img in images]

In [10]:
prompt = (
    f"You are given {len(images_base64)} images represented as base64 strings, "
    f"and a description: '{request.description}'. Analyze the images and select the most relevant "
    f"one based on the description. Respond with the index (0-based) of the chosen image."
)

NameError: name 'request' is not defined

In [12]:
from openai import OpenAI
from schemas import SelectedImageIndex,FindImage

# Step 3: Prepare prompt and make API call
prompt = (
    f"You are given {len(images_base64)} images represented as base64 strings, "
    f"and a description: '{query}'. Analyze the images and select the most relevant "
    f"one based on the description. Respond with the index (0-based) of the chosen image."
)

client = OpenAI()  # Instantiate OpenAI client
response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                *[
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{img_b64}"},
                    }
                    for img_b64 in images_base64
                ],
            ],
        }
    ],
    response_format=SelectedImageIndex,  # Enforce the integer response schema
)

# Parse the index from the response
chosen_index = response.index

AttributeError: 'ParsedChatCompletion[SelectedImageIndex]' object has no attribute 'index'

In [14]:
chosen_index = response.choices[0].message.parsed.index
chosen_index

1

In [37]:
articles = """
title = EV targets 'will not be weakened' despite pressure
content = Rules on the percentage of electric vehicles (EVs) car firms must sell will not be weakened, the transport secretary has said, despite growing pressure from the industry.

The mandate will become tougher next year ahead of a complete ban on new diesel and petrol cars in 2035, but car makers have suggested the rules could threaten jobs.

Several senior government ministers and car makers with factories in the UK will hold crunch talks this week about the EV mandate and the slump in car demand.

Louise Haigh said she will look at "flexibilities", but insisted that "the mandate will not be weakened".

"There has been a downturn in demand on a global level so we are absolutely in listening mode - we want to discuss how the current situation is affecting them, but we are not diluting our ambition," the transport secretary told LBC Radio on Sunday.

"I’m meeting with Nissan tomorrow and the business secretary, the energy minister and I are meeting with a number of automotive manufacturers later in the week in order to discuss the challenges that they face on a global scale."

*Industry 'will likely miss' target*
According to the mandate, EVs must make up 22% of a firm's car sales and 10% of their van sales this year. For every car sale that pushes it outside of that mandate, they must pay a £15,000 fine.

There are some existing workarounds in the rules allowing firms to avoid the fines. They can, for example, buy credits from companies which have EV sales above 22% of their total sales.

"There are flexibilities available to them in the current mandate," Haigh told LBC.

"We want to make sure that the flexibilities are appropriate for them... but we absolutely don’t water down our ambition."

The rules get tougher every year ahead of a complete ban of new petrol and diesel car sales by 2035, though Labour said in its manifesto it would bring this forward to 2030 - reinstating the old target - as part of its wider commitments to climate change policy.

The trade association, Society of Motor Manufacturers and Traders (SMMT), has said the industry "will likely miss" this year's targets - calculating that around 18% of UK car sales are currently EVs.

SMMT and car makers say they support the government's long-term aims, but they are looking for some concessions in this week's talks.

One suggestion is that the penalties car makers have to pay for missing the targets should be reduced, the BBC understands.

Meanwhile, the SMMT is calling for government grants for buyers of EVs and tax changes.

It says changes are needed to deal with the plummet in demand, with its data showing that new car registrations in the UK remain a fifth lower than pre-Covid.

A Nissan spokesperson told the BBC it is “committed to playing a full role in the transition to net zero in the UK", but said "changes are required now to reflect the reality of the market, as we continue to work together to encourage more drivers to make the switch”.

A spokesperson for Stellantis - which owns Citroën, Peugeot, Vauxhall and several other car brands - said it is aligned with the objectives of the mandate.

But it added: "To remain efficient in the UK, we are strategically reviewing our operations working with our union partners as previously announced."

In April, Stellantis chief executive Carlos Tavares told The Telegraph newspaper the rules were "terrible" and that it could force the car maker to reduce its presence in the UK.

The Unite union has urged the company to commit to its future at its Luton and Ellesmere Port factories to address the "fear and rumour" it has created through announcing the strategic review.

Unite added it is "already having constructive discussions with government and industry to reform the EV mandate to protect jobs".
"""

In [53]:
import requests

# API endpoint
url = "http://127.0.0.1:8000/generate-image-query/"

# Request payload
payload = {
    "articles": articles,  # Replace with your desired description
    "image_url": ''  # Number of images to analyze
}

# Request headers
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)
print(response)

<Response [200]>


In [54]:
response.json()['search_query']

'Search query: "Electric vehicle mandate UK car industry regulations"'

In [59]:
import requests

# API endpoint
url = "http://127.0.0.1:8000/generate-full-article/"

# Request payload
payload = {
    "articles": articles,  # Replace with your desired description
    "image_url": ''  # Number of images to analyze
}

# Request headers
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)
print(response)

<Response [200]>


In [62]:
response.json()['article']

'```markdown\n# UK Government Stands Firm on EV Targets Amid Industry Turmoil\n\n![image](https://static.euronews.com/articles/wires/826/34/8263452/1200x675_ltxb0p.jpg)\n\nThe UK\'s electric vehicle (EV) mandate, which stipulates the percentage of electric vehicles that manufacturers must sell, remains untouched despite mounting pressure from the automotive industry. As the government braces for critical discussions with car makers, Transport Secretary Louise Haigh has reaffirmed that these EV targets will not be weakened, despite global challenges affecting vehicle demand.\n\n## Industry Pressures and Government Response\n\nWith a global slump in car demand, manufacturers have expressed concerns that the stringent mandates could threaten jobs and hinder business operations. Car producers like Nissan and those under Stellantis, which owns brands such as Citroën and Vauxhall, align with the government\'s long-term climate goals but urge for immediate rule modifications to address curren